# Task 1 of Assignment 2

## Support Vector Machine

**Objective:** Process the MIMIC chest X-ray dataset as a single entity, training all pathologies together.

**Split Ratio:** 5:3:2 (50% training, 30% validation, 20% testing)

**Pathology Grouping:** No grouping, all 13 pathologies (Atelectasis, Cardiomegaly, etc.) treated uniformly

**Key Features:**
- Data split with fixed seed (seed=42) for consistency
- NaN values converted to 0, no filtering of all-NaN samples
- Outputs three datasets: train_dataset, val_dataset, test_dataset

### 1. Load Data (All Label)

In [76]:
import os
import numpy as np
import pandas as pd
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#import tensorflow as tf
#import matplotlib.pyplot as plt
#import seaborn as sns
import datetime
import joblib

from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve
#from tqdm import tqdm
#from torch.optim import AdamW
#from torch.optim.lr_scheduler import ReduceLROnPlateau
#from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
#from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, multilabel_confusion_matrix, make_scorer, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

In [2]:
# Set TensorFlow logging level to suppress unnecessary output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')

In [3]:
# Helper function: Load embeddings from TFRecord files
def load_embedding(embedding_path):
    raw_dataset = tf.data.TFRecordDataset([embedding_path])
    for raw_record in raw_dataset.take(1):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())
        embedding_feature = example.features.feature['embedding']
        embedding_values = embedding_feature.float_list.value
    embedding = torch.tensor(embedding_values)
    if torch.isnan(embedding).any() or torch.isinf(embedding).any():
        raise ValueError(f"NaN or Inf found in embedding: {embedding_path}")
    return embedding

In [4]:
# Custom dataset class for MIMIC chest X-ray embeddings
class MIMIC_Embed_Dataset(Dataset):
    pathologies = [
        "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", "Lung Lesion",
        "Edema", "Consolidation", "Pneumonia", "Atelectasis", "Pneumothorax",
        "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"
    ]

    def __init__(self, embedpath, csvpath, metacsvpath, views=["PA"], mode="train", seed=42, split_ratio=[0.5, 0.3, 0.2], unique_patients=True):
        super().__init__()
        np.random.seed(seed)  # Set seed for reproducibility
        self.pathologies = sorted(self.pathologies)
        self.mode = mode
        self.embedpath = embedpath
        self.csv = pd.read_csv(csvpath)
        self.metacsv = pd.read_csv(metacsvpath)
        self.split_ratio = split_ratio

        # Merge CSV data
        self.csv = self.csv.set_index(["subject_id", "study_id"])
        self.metacsv = self.metacsv.set_index(["subject_id", "study_id"])
        self.csv = self.csv.join(self.metacsv).reset_index()

        self.csv["view"] = self.csv["ViewPosition"]
        self.limit_to_selected_views(views)

        if unique_patients:
            self.csv = self.csv.groupby("subject_id").first().reset_index()

        # Split dataset into train, validation, and test sets
        n_row = self.csv.shape[0]
        if self.mode == "train":
            self.csv = self.csv[:int(n_row * self.split_ratio[0])]
        elif self.mode == "valid":
            self.csv = self.csv[int(n_row * self.split_ratio[0]):int(n_row * (self.split_ratio[0] + self.split_ratio[1]))]
        elif self.mode == "test":
            self.csv = self.csv[int(n_row * (self.split_ratio[0] + self.split_ratio[1])):]
        else:
            raise ValueError(f"mode must be one of [train, valid, test], got {self.mode}")

        # Process labels
        healthy = self.csv["No Finding"] == 1
        labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                self.csv.loc[healthy, pathology] = 0
                mask = self.csv[pathology]
            labels.append(mask.values)
        self.labels = np.asarray(labels).T.astype(np.float32)
        self.labels[np.isnan(self.labels)] = 0  # Replace all NaN values with 0
        self.labels = np.where((self.labels != 0) & (self.labels != 1), 0, self.labels)

        self.csv = self.csv.reset_index(drop=True)
        self.pathologies = list(np.char.replace(self.pathologies, "Pleural Effusion", "Effusion"))
        self.csv["offset_day_int"] = self.csv["StudyDate"]
        self.csv["patientid"] = self.csv["subject_id"].astype(str)

    def limit_to_selected_views(self, views):
        # Limit dataset to specified views
        if not isinstance(views, list):
            views = [views]
        if '*' in views:
            views = ["*"]
        self.views = views
        self.csv["view"] = self.csv["view"].fillna("UNKNOWN")
        if "*" not in views:
            self.csv = self.csv[self.csv["view"].isin(self.views)]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retrieve a single sample from the dataset
        sample = {}
        sample["idx"] = idx
        sample["lab"] = self.labels[idx]
        subjectid = str(self.csv.iloc[idx]["subject_id"])
        studyid = str(self.csv.iloc[idx]["study_id"])
        dicom_id = str(self.csv.iloc[idx]["dicom_id"])
        embed_file = os.path.join(self.embedpath, f"p{subjectid[:2]}", f"p{subjectid}", f"s{studyid}", f"{dicom_id}.tfrecord")
        sample["embedding"] = load_embedding(embed_file)
        return sample

In [5]:
# Function to initialize datasets with given paths, seed, and split ratio
def initialize_mimic_datasets(embedpath, csvpath, metacsvpath, seed=42, split_ratio=[0.5, 0.3, 0.2]):
    """
    Initialize MIMIC chest X-ray datasets with specified paths, seed, and split ratio.
    
    Args:
        embedpath (str): Path to the embedding files
        csvpath (str): Path to the CheXpert CSV file
        metacsvpath (str): Path to the metadata CSV file
        seed (int): Random seed for reproducibility (default: 42)
        split_ratio (list): Train, validation, test split ratio (default: [0.5, 0.3, 0.2])
    
    Returns:
        tuple: (train_dataset, val_dataset, test_dataset)
    """
    train_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="train",
        seed=seed,
        split_ratio=split_ratio
    )
    val_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="valid",
        seed=seed,
        split_ratio=split_ratio
    )
    test_dataset = MIMIC_Embed_Dataset(
        embedpath=embedpath,
        csvpath=csvpath,
        metacsvpath=metacsvpath,
        mode="test",
        seed=seed,
        split_ratio=split_ratio
    )
    return train_dataset, val_dataset, test_dataset

In [6]:
# Data paths
embedpath = "C:/Users/idaul/Dropbox/1 Idauli/1 NUS PhD/0 2025 Sem 5/1 SPH6004 Advanced Statistical Learning/Assignments/Assignment 2/Data/sph6004/generalized-image-embeddings-for-the-mimic-chest-x-ray-dataset-1.0/files"
csvpath = "C:/Users/idaul/Dropbox/1 Idauli/1 NUS PhD/0 2025 Sem 5/1 SPH6004 Advanced Statistical Learning/Assignments/Assignment 2/Data/mimic-cxr-2.0.0-chexpert.csv"
metacsvpath = "C:/Users/idaul/Dropbox/1 Idauli/1 NUS PhD/0 2025 Sem 5/1 SPH6004 Advanced Statistical Learning/Assignments/Assignment 2/Data/mimic-cxr-2.0.0-metadata.csv"

workpath = "C:/Users/idaul/Dropbox/1 Idauli/1 NUS PhD/0 2025 Sem 5/1 SPH6004 Advanced Statistical Learning/Assignments/Assignment 2/Data/"

In [7]:
train_dataset, val_dataset, test_dataset = initialize_mimic_datasets(
    embedpath=embedpath,
    csvpath=csvpath,
    metacsvpath=metacsvpath,
    seed=42,
    split_ratio=[0.5, 0.3, 0.2]
)

In [9]:
print("Length of training dataset:", len(train_dataset), "  Validation dataset:", len(val_dataset), "  Test dataset:", len(test_dataset))

Length of training dataset: 22814   Validation dataset: 13688   Test dataset: 9126


In [18]:
len(train_dataset[1000]['embedding'])

1376

In [10]:
train_dataset[1000]

{'idx': 1000,
 'lab': array([0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0.], dtype=float32),
 'embedding': tensor([-0.6009, -2.6448,  1.1589,  ...,  0.3574,  1.4271, -2.1083])}

In [13]:
val_dataset[1000]

{'idx': 1000,
 'lab': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'embedding': tensor([ 0.0728, -1.7532,  0.8360,  ..., -0.7722,  0.8441, -0.5825])}

In [11]:
test_dataset[1000]

{'idx': 1000,
 'lab': array([1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32),
 'embedding': tensor([-0.5893, -1.4492,  1.0979,  ..., -0.6514,  1.0990, -0.9476])}

In [14]:
type(train_dataset)

__main__.MIMIC_Embed_Dataset

In [19]:
# Convert the training datasets to numpy arrays for scikit-learn 
X_train = np.stack([sample['embedding'].numpy() for sample in train_dataset])  # shape: [n_samples, embedding_dim]
y_train = np.stack([sample['lab'] for sample in train_dataset])               # shape: [n_samples, n_labels]

In [20]:
#Check
X_train.shape

(22814, 1376)

In [21]:
#Check
y_train.shape

(22814, 13)

In [23]:
type(X_train)

numpy.ndarray

In [27]:
X_train[0]

array([ 0.12571079, -1.8030074 ,  1.2842511 , ..., -0.7076767 ,
        1.0859656 ,  0.02559637], dtype=float32)

In [28]:
# Save the numpy array to CSVs
np.savetxt(workpath+"X_train.csv", X_train, delimiter=',')
np.savetxt(workpath+"y_train.csv", y_train, delimiter=',')

### Train SVM

In [60]:
#To start, set C values and kernel type
params_svm = {'estimator__kernel':['poly','rbf'],
              'estimator__C':[0.01, 0.1, 1, 10]
             }
params_svm

{'estimator__kernel': ['poly', 'rbf'], 'estimator__C': [0.01, 0.1, 1, 10]}

In [61]:
# Define a scoring metric for multiclass/multilabel classification
# F1 score balances precision and recall
f1_scorer = make_scorer(f1_score, average='micro') # 'micro' is suitable for multilabel

In [62]:
# Since it's a large dataset, use RandomizedSearch instead of the typical GridSearch
# Start with 5 splits for cross validation (default)

SVMModel = OneVsRestClassifier(SVC(random_state=42))
SVMSearch = RandomizedSearchCV(
    SVMModel,
    param_distributions=params_svm,
    scoring=f1_scorer,
    n_iter=4,
    cv=5
)

#time
print(datetime.datetime.now().strftime("%H:%M:%S"))

# Call the search and record the best parameters
BestSVM = SVMSearch.fit(X_train, y_train)

#time
print(datetime.datetime.now().strftime("%H:%M:%S"))

04:06:40
12:55:56


In [66]:
joblib.dump(BestSVM, workpath+"BestSVM_AllData.pkl")

['C:/Users/idaul/Dropbox/1 Idauli/1 NUS PhD/0 2025 Sem 5/1 SPH6004 Advanced Statistical Learning/Assignments/Assignment 2/Data/BestSVM_AllData.pkl']

In [67]:
#Inspect
BestSVM

RandomizedSearchCV(cv=5,
                   estimator=OneVsRestClassifier(estimator=SVC(random_state=42)),
                   n_iter=4,
                   param_distributions={'estimator__C': [0.01, 0.1, 1, 10],
                                        'estimator__kernel': ['poly', 'rbf']},
                   scoring=make_scorer(f1_score, response_method='predict', average=micro))

#### Prep the validation and test datasets

In [68]:
# Convert the validation datasets to numpy arrays for scikit-learn 
X_val = np.stack([sample['embedding'].numpy() for sample in val_dataset])  # shape: [n_samples, embedding_dim]
y_val = np.stack([sample['lab'] for sample in val_dataset])               # shape: [n_samples, n_labels]

In [69]:
# Save the numpy array to CSVs
np.savetxt(workpath+"X_val.csv", X_train, delimiter=',')
np.savetxt(workpath+"y_val.csv", y_train, delimiter=',')

In [70]:
# Convert the test datasets to numpy arrays for scikit-learn 
X_test = np.stack([sample['embedding'].numpy() for sample in test_dataset])  # shape: [n_samples, embedding_dim]
y_test = np.stack([sample['lab'] for sample in test_dataset])               # shape: [n_samples, n_labels]

In [71]:
# Save the numpy array to CSVs
np.savetxt(workpath+"X_test.csv", X_train, delimiter=',')
np.savetxt(workpath+"y_test.csv", y_train, delimiter=',')

#### Test on the validation set

In [72]:
y_pred_val = BestSVM.predict(X_val)

In [73]:
# Print the classification report
print(classification_report(y_val, y_pred_val, zero_division=0, target_names=val_dataset.pathologies))

                            precision    recall  f1-score   support

               Atelectasis       0.54      0.08      0.14      1211
              Cardiomegaly       0.56      0.09      0.15      1085
             Consolidation       0.00      0.00      0.00       243
                     Edema       0.52      0.07      0.12       356
Enlarged Cardiomediastinum       0.00      0.00      0.00       189
                  Fracture       0.67      0.01      0.01       292
               Lung Lesion       0.00      0.00      0.00       392
              Lung Opacity       0.63      0.05      0.09      1612
                  Effusion       0.67      0.44      0.53      1121
             Pleural Other       0.00      0.00      0.00       123
                 Pneumonia       0.86      0.01      0.02       685
              Pneumothorax       0.50      0.02      0.03       175
           Support Devices       0.46      0.06      0.11       572

                 micro avg       0.62      0.1

In [77]:
#Accuracy
print(accuracy_score(y_val, y_pred_val))

0.7128872004675628


#### Test on test dataset

In [78]:
y_pred_test = BestSVM.predict(X_test)

In [79]:
# Print the classification report
print(classification_report(y_test, y_pred_test, zero_division=0, target_names=test_dataset.pathologies))

                            precision    recall  f1-score   support

               Atelectasis       0.55      0.08      0.14       855
              Cardiomegaly       0.57      0.08      0.15       761
             Consolidation       0.00      0.00      0.00       166
                     Edema       0.52      0.06      0.11       229
Enlarged Cardiomediastinum       0.00      0.00      0.00       139
                  Fracture       0.00      0.00      0.00       184
               Lung Lesion       0.00      0.00      0.00       271
              Lung Opacity       0.52      0.04      0.07      1120
                  Effusion       0.67      0.50      0.57       742
             Pleural Other       0.00      0.00      0.00        81
                 Pneumonia       0.75      0.01      0.03       431
              Pneumothorax       0.00      0.00      0.00       105
           Support Devices       0.46      0.07      0.13       398

                 micro avg       0.61      0.1

In [80]:
#Accuracy
print(accuracy_score(y_test, y_pred_test))

0.7060048213894368
